### Image Segmentation
Image segmentation is a fundamental problem in computer vision where the goal is to group pixels with similar characteristics such as color, intensity, texture, semantic class. The result of image segmentation is a set of segments that collectively cover the entire image. Pixels grouped in the same segments are similar with respect to some characteristic. Adjacent regions are significantly different with respect to the same characteristic. 

In this exercise class we will focus on the problem of image segmentation using low-level features like texture and color. The goal is to produce a set of segments where pixels grouped in the same segment have similar texture or color.

In [ ]:
# run this before starting the lab
import numpy as np
import cv2 as cv
import glob
import time
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import scipy.io
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries

In [ ]:
# we are going to use this to save all the segmentation obtained in this lab to display them at the end
history_segmentation = {}
def save_history(image_path, segmentation, method_name):
    if image_path not in history_segmentation: 
        history_segmentation[image_path] = {}
    history_segmentation[image_path][method_name] = segmentation 

In [ ]:
def visualize_filters(filters, num_rows, num_cols, theta_list=None, lambda_list=None):
    """
    Visualize the filters.
    :param num_rows. How many rows are used to display the filters.
    :param num_cols. How many columns are used to display the filters. 
    :param filters. Filters with the shape N x K x K.
    :param theta_list. The list with the theta parameters.
    :param lambda_list. The list with the lambda paramters.
    """
    plt.figure(figsize=(15, 15)) 
 
    for i in range(num_rows):
        for j in range(num_cols):
            if i * num_cols + j < len(filters):
                plt.subplot(num_rows, num_cols, i * num_cols + j + 1)
                plt.axis('off')                  
                plt.imshow(filters[i * num_cols + j], extent=[0, 100, 0, 1], aspect=100)
                if theta_list is not None:
                    plt.title("L = %s T= %.2f" % (lambda_list[i], theta_list[j]))  

In [ ]:
def show_images(image_1, image_2=None, image_3=None, name_1='image_1', name_2='image_2', name_3='image_3', time_out=0):
    """
    Show images received as parameters.
    """
    cv.imshow(name_1, image_1)
    if image_2 is not None:
        cv.imshow(name_2, image_2)
    if image_3 is not None:
        cv.imshow(name_3, image_3)
    cv.waitKey(time_out)
    cv.destroyAllWindows()

In [ ]:
NUMBER_DIFFERENT_COLORS = 50
random_colors = np.uint8(np.random.randint(255, size=(NUMBER_DIFFERENT_COLORS, 3)))

def label_to_rgb(img_label, colors=random_colors):
    """
    It colors the image based on the label number.
    :param img_label. The 'image' containing a label for each patch/portion of the image.
    :param colors
    :return the colored image.
    """
    h, w = img_label.shape
    fake_colored_label_image = np.uint8(np.zeros((h, w, 3))) 
    
    number_labels = img_label.max() + 1
    number_different_colors = colors.shape[0] 
    
    for label in range(number_labels):
        rows, columns = np.where(img_label == label)
        fake_colored_label_image[rows, columns, :] = colors[label % number_different_colors]
    return fake_colored_label_image

In [ ]:
def apply_kmeans(features, num_clusters):
    """
    Return the model after performing the Kmeans algorithm.
    :param features
    :param num_clusters.
    :return kmeans model.
    """
    kmeans_model = KMeans(init="random", n_clusters=num_clusters, n_init=10, max_iter=300, random_state=42)
    begin = time.time() 
    kmeans_model.fit(np.float32(features))
    end = time.time() 
    print(f"Kmeans clustering took {end-begin} seconds")
    return kmeans_model

In [ ]:
def build_gabor_filters(theta_list, lambda_list) -> list:
    # useful links for gabor filters
    # build Gabor filters for texture 
    # see the gif Gabor_filter_values_visualization.gif 
    # https://medium.com/@anuj_shah/through-the-eyes-of-gabor-filter-17d1fdb3ac97
    # https://stackoverflow.com/questions/30071474/opencv-getgaborkernel-parameters-for-filter-bank
    """
    We build Gabor filters with the window size of 31 pixels and theta and lambda received in the paramters.
    :param theta_list. The list with the theta parameters.
    :param lambda_list. The list the lambda paramters.
    :return. list of Gabor filters.
    """
    filters = []
    ksize = 31
    
    for lambda_ in lambda_list:   
        for theta in theta_list:    
            kern = cv.getGaborKernel((ksize, ksize), 4, theta, lambda_, 0.5, 0, ktype=cv.CV_32F)
            kern /= 1.5 * kern.sum()
            filters.append(kern)            
    return filters


def apply_filters(img, filters: list):
    """
    :param img - the image on which we apply the filters.
    :param filter - a list with filters.
    :return. The results of applying the filters on the image.
    """
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    h, w = img_gray.shape
    
    number_filters = len(filters)
    responses = np.zeros((h, w, number_filters))
    for idx, kernel in enumerate(filters):        
        responses[:, :, idx] = np.abs(cv.filter2D(img_gray, cv.CV_32F, kernel))        
    return responses

# Pixel level segmentation
## Pixel level segmentation - texture

###  Visualize the Gabor filters

In [ ]:
# We build Gabor filters with the window size of 31 pixels and theta between 0 and PI (with a step of 45 degrees)
# and lambda between 10 and 80 (inclusive) with a step of 20.
theta_list = np.arange(0, np.pi, np.pi / 4)
lambda_list = np.arange(10, 90, 20)
gabor_filters = build_gabor_filters(theta_list=theta_list, lambda_list=lambda_list)
visualize_filters(gabor_filters, num_rows=len(lambda_list), num_cols=len(theta_list),
                        theta_list=theta_list, lambda_list=lambda_list)

### Our first task is to segment the images from the 'dataset' folder using the Gabor / Custom filters response for each pixel as features. 
In order to complete the task, we have to do:
1. Obtain the filters response (our features), for each pixel in our dataset.
2. Cluster the resulting features using the K-means algorithm.
3. Obtain the segmentation of each image by predicted the cluster assigment of the pixels.

In [ ]:
def get_filters_response(image_paths: list, filters):
    """
    Return a matrix of size (number_of_total_pixels X number_of_applied_filters) which is defined
    by applying the filters on each pixel in our dataset.
    1. For each image from the dataset, apply the filters.
    2. Store the result into a matrix of dimension (number_of_total_pixels X number_of_applied_filters).
    :param image_paths: The paths of the images in our dataset.
    :param filters: The filters used to define each pixel.
    :return: Return a matrix of size (number_of_total_pixels X number_of_applied_filters).
    """
    features = []
    # TODO: 
    return np.array(features)

In [ ]:
def get_segmentation_based_on_texture(image, filters, kmeans_model):
    """
    1. Apply the filters on the image.
    2. Obtain the cluster number of each pixel.
    3. Obtain the segmentation by reshaping the kmeans output.
    """
    # TODO: 
    
    segmentation = ...
    return segmentation

In [ ]:
def segment_images_using_texture_filters(image_paths, filters, num_clusters=5, method_name='gabor_filters'):
    """
    1. Get the filters response for each pixel in our dataset.
    2. Cluster the features using the Kmeans algorithm.
    3. Get the segmetation of the image using the clusters obtained earlier.
    4. Visualize the result (use the show_images function)
    """
    
    features = get_filters_response(image_paths, filters)
    print(f'Our features size is {features.shape}')
    kmeans_model = apply_kmeans(features, num_clusters=num_clusters)
    for image_path in image_paths:
        img = cv.imread(image_path)
        segmentation = get_segmentation_based_on_texture(img, filters, kmeans_model)
        colored_segmentation = label_to_rgb(segmentation)
        # save the output
        save_history(image_path, colored_segmentation, method_name)
        show_images(image_1=img, name_1='img', image_2=colored_segmentation, name_2='segmentation')
    

In [ ]:
# run segmentation by texture
image_paths = glob.glob("dataset/*_s.bmp")
segment_images_using_texture_filters(image_paths, gabor_filters)

#### Now, we are going to change the filters and see what segmantion maps we will get

In [ ]:
def load_custom_filters():
    """
    Load filters from Matlab and return the filters as a list.
    """
    mat = scipy.io.loadmat('colectieFiltre.mat') 
    filters = mat['F']
    filters_list = []
    for i in range(filters.shape[2]):
        filters_list.append(filters[:, :, i].copy())
    return filters_list

In [ ]:
# Let's load and visualize the new filters
custom_filters = load_custom_filters()
visualize_filters(custom_filters, num_rows=7, num_cols=6)

In [ ]:
# run segmentation by texture
image_paths = glob.glob("dataset/*_s.bmp")
segment_images_using_texture_filters(image_paths, custom_filters, method_name='custom_filters')

## Pixel level segmentation - color
### Our second task is to segment the images from the 'dataset' folder using the RBG values as features for each pixel. 
In order to complete the task, we have to do:
1. Get the RGB value, for each pixel in our dataset.
2. Cluster the resulting features using the K-means algorithm.
3. Obtain the segmentation of each image by predicted the cluster assigment of the pixels.

In [ ]:
def get_rgb_response(image_paths: list):
    """
    Return a matrix of size (number_of_total_pixels X 3).
    1. For each image from the dataset, get the BGR/RGB pixel values.
    2. Store the result into a matrix of dimension (number_of_total_pixels X 3).
    :param image_paths: The paths of the images in our dataset.
    :param filters: The filters used to define each pixel.
    :return: Return a matrix of size (number_of_total_pixels X 3).
    """
    features = [] 
    # TODO:
    return np.array(features)

In [ ]:
def get_segmentation_based_on_color(image, kmeans_model):
    """
    1. Get the RGB for each pixel in the image.
    2. Obtain the cluster number of each pixel.
    3. Obtain the segmentation by reshaping the kmeans output.
    """ 
    
    # TODO: 
    segmentation = ...   
    return segmentation

In [ ]:
def segment_images_based_on_color(image_paths, num_clusters=5, method_name="color"):
    """
    1. Get the RGB for each pixel in our dataset.
    2. Cluster the features using the Kmeans algorithm.
    3. Get the segmatation of the image using the clusters obtained earlier.
    4. Visualize the result (use the show_images function)
    """
    
    features = get_rgb_response(image_paths)
    print(f'Our features size is {features.shape}')
    kmeans_model = apply_kmeans(features, num_clusters=num_clusters)
    for image_path in image_paths:
        img = cv.imread(image_path)
        segmentation = get_segmentation_based_on_color(img, kmeans_model)
        colored_segmentation = label_to_rgb(segmentation)
        # save the output
        save_history(image_path, colored_segmentation, method_name)
        show_images(image_1=img, name_1='img', image_2=colored_segmentation, name_2='segmentation')

In [ ]:
# run segmentation by color
image_paths = glob.glob("dataset/*_s.bmp")
segment_images_based_on_color(image_paths)

# Superpixel level segmentation
A superpixel can be defined as a group of pixels which have similar characteristics. It is generally color based segmentation. Superpixels can be very helpful for image segmentation: 
(i)   superpixels reduce the complexity of the images themselves from hundreds of thousands of pixels to only a few hundred superpixels providing thus an efficient representation of images
(ii)  pixels that belong to a superpixel group share some sort of commonality, such as similar color or texture distribution.
(iii) most superpixel algorithms oversegment the image. This means that most of important boundaries in the image are found; however, at the expense of generating many insignificant boundaries. 

#### We are going to use the SLIC (Simple Linear Iterative Clustering) algorithm from Scikit-image.
more details you can find here: https://scikit-image.org/docs/dev/api/skimage.segmentation.html#skimage.segmentation.slic

In [ ]:
def apply_slic(image, show_images_=False):
    """
    Apply the SLIC algorithm on the image received as paramters.
    :param image.
    :show_images_. If True the result of the algorithm is shown.
    :return The superpixels obtained by running the SLIC algorithm.
    """
    segments_slic = slic(image, n_segments=250, compactness=10, start_label=1)  
    if show_images_:
        colored_segments = label_to_rgb(segments_slic)
        show_images(image_1=image, image_2=colored_segments, name_1='image', name_2='splic_segmentation',
                    image_3=mark_boundaries(image, segments_slic), name_3='boundaries')
    return segments_slic

In [ ]:
# let's see how superpixels works
image = cv.imread('dataset/7_25_s.bmp')
apply_slic(image, show_images_=True)

### Our third task is to segment the images from the 'dataset' folder using the Gabor filters response for each superpixel as features. 
In order to complete the task, we have to do:
1. Get the superpixel for each image.
2. For each superpixel, obtain the mean of the filters response (these are the features). 
2. Cluster the resulting features using the K-means algorithm.
3. Obtain the segmentation of each image by predicted the cluster assigment of the pixels.

In [ ]:
def get_mean_response_superpixel_texture(image, segments_slic, filters):
    """
    1. For each superpixel, obtain the mean of the filters response (these are the features).
    :param image.
    :param segments_slic: The results obtained after running the SLIC algorithm.
    :param filters. The filters applied to the pixels.
    :return The mean filters response for each superpixels.
    """
    responses = apply_filters(image, filters)
    number_superpixels = segments_slic.max()
    superpixel_mean_response = np.zeros((number_superpixels, len(filters)))
    for label in range(number_superpixels):
        rows, columns = np.where(segments_slic == label + 1) # this is because when we called the splic function, we set start_label=1
        superpixel_responses = responses[rows, columns, :]
        superpixel_mean_response[label] = np.mean(superpixel_responses, axis=0) 
        
    return superpixel_mean_response

In [ ]:
def get_superpixel_segmetation_based_on_texture(image, filters, kmeans_model):
    """
    1. For each image, obtain the superpixels. 
    2. For each superpixel, obtain the mean of the filters response.
    3. Cluster the resulting features using the K-means algorithm.
    4. Get the segmentation map by labelling each superpixel with the corresponding cluster label.
    """
    segments_slic = apply_slic(image)
    superpixel_mean_response = get_mean_response_superpixel_texture(image, segments_slic, filters)
    num_superpixel, _ = superpixel_mean_response.shape
    predicted_labels = kmeans_model.predict(superpixel_mean_response)
    segmentation = np.zeros_like(segments_slic)
    for label in range(num_superpixel): 
        rows, columns = np.where(segments_slic == label + 1)    
        segmentation[rows, columns] = predicted_labels[label]
    return segmentation
    

In [ ]:
def segment_images_with_superpixel_based_on_texture(image_paths, filters, num_clusters=5, method_name="superpixel_gabor"):
    """
    1. Get the superpixel for each image.
    2. For each superpixel, obtain the mean of the filters response (these are the features). 
    2. Cluster the resulting features using the K-means algorithm.
    3. Obtain the segmentation of each image by predicted the cluster assigment of the pixels.
    """
    def get_superpixels_features(images_paths_, filters_):
        features_ = []
        num_filters_ = len(filters_)
        for image_path in images_paths_:
            image = cv.imread(image_path)
            segments_slic = apply_slic(image)
            superpixel_mean_response = get_mean_response_superpixel_texture(image, segments_slic, filters_)
            features_.extend(superpixel_mean_response.reshape(-1, num_filters_))
        return np.array(features_)
    
    features = get_superpixels_features(image_paths, filters)
    print(f'Our features size is {features.shape}')
    
    kmeans_model = apply_kmeans(features, num_clusters=num_clusters)
    
    for image_path in image_paths:
        img = cv.imread(image_path)
        segmentation = get_superpixel_segmetation_based_on_texture(img, filters=filters, kmeans_model=kmeans_model)
        colored_segmentation = label_to_rgb(segmentation)
        # save the output
        save_history(image_path, colored_segmentation, method_name)
        show_images(image_1=img, name_1='img', image_2=colored_segmentation, name_2='segmentation')

In [ ]:
# run segmentation with superpixels based on texture
image_paths = glob.glob("dataset/*_s.bmp")
theta_list = np.arange(0, np.pi, np.pi / 4)
lambda_list = np.arange(10, 90, 20)
gabor_filters = build_gabor_filters(theta_list=theta_list, lambda_list=lambda_list)
segment_images_with_superpixel_based_on_texture(image_paths, gabor_filters)

# Superpixel level segmentation - color
### Our fourth (and last) task is to segment the images from the 'dataset' folder using the RGB mean values for each superpixel as features. 
In order to complete the task, we have to do:
1. Get the superpixel for each image.
2. For each superpixel, obtain the mean rgb value (these are the features). 
2. Cluster the resulting features using the K-means algorithm.
3. Obtain the segmentation of each image by predicted the cluster assigment of the pixels.

In [ ]:
def get_mean_color_superpixel_response(image, segments_slic):    
    """
    For each superpixel, obtain the mean rgb value.
    :param image.
    :param segments_slic. The result obtained by running the SLIC algorith.
    :return. The mean rgb value for each superpixel in the image.
    """
    number_superpixels = segments_slic.max()
    superpixel_mean_color = np.zeros((number_superpixels, 3))
    
    # TODO:
    return superpixel_mean_color

In [ ]:
def get_superpixel_segmetation_based_on_color(image, kmeans_model):
    """
    1. For each image, obtain the superpixels. 
    2. For each superpixel, obtain the mean rgb value.
    3. Cluster the resulting features using the K-means algorithm.
    4. Get the segmentation map by labelling each superpixel with the corresponding cluster label.
    """
    # TODO:
    segments_slic = ...
    
    segmentation = np.zeros_like(segments_slic)
     
    return segmentation

In [ ]:
def segment_images_with_superpixel_based_on_color(image_paths, num_clusters=5, method_name="superpixel_color"):
    """
    1. Get the superpixel for each image.
    2. For each superpixel, obtain the mean of the rgb values (these are the features). 
    2. Cluster the resulting features using the K-means algorithm.
    3. Obtain the segmentation of each image by predicted the cluster assigment of the pixels.
    """
    def get_superpixels_features(images_paths_):
        features_ = [] 
        for image_path in images_paths_:
            image = cv.imread(image_path)
            segments_slic = apply_slic(image)
            superpixel_mean_response = get_mean_color_superpixel_response(image, segments_slic)
            features_.extend(superpixel_mean_response.reshape(-1, 3))
        return np.array(features_)
    
    features = get_superpixels_features(image_paths)
    print(f'Our features size is {features.shape}')
    
    kmeans_model = apply_kmeans(features, num_clusters=num_clusters)
    
    for image_path in image_paths:
        img = cv.imread(image_path)
        segmentation = get_superpixel_segmetation_based_on_color(img, kmeans_model=kmeans_model)
        colored_segmentation = label_to_rgb(segmentation)
        # save the output
        save_history(image_path, colored_segmentation, method_name)
        show_images(image_1=img, name_1='img', image_2=colored_segmentation, name_2='segmentation')

In [ ]:
# run segmentation with superpixels based on color
image_paths = glob.glob("dataset/*_s.bmp") 
segment_images_with_superpixel_based_on_color(image_paths)

In [ ]:
history_segmentation.keys()

In [ ]:
def show_segmentation_history():
    """
    Display all the segmatation obtained in this lab.
    """
    global history_segmentation
    image_paths = list(history_segmentation.keys())
    num_rows = len(image_paths)
    num_cols = len(history_segmentation[image_paths[0]].keys()) + 2
    
    plt.figure(figsize=(50, 50))  
                
    for idx_row, image_path in enumerate(image_paths):
        image = cv.imread(image_path) 
        gt_path = image_path.replace(".bmp", "_HQGT.bmp")
        gt = cv.imread(gt_path)
        
        # plot 
        plt.subplot(num_rows, num_cols, idx_row * num_cols + 1)
        plt.axis('off')                  
        plt.imshow(image[:, :, [2, 1, 0]])
        plt.title('image')
        
        plt.subplot(num_rows, num_cols, idx_row * num_cols + 2)   
        plt.axis('off') 
        plt.imshow(gt[:, :, [2, 1, 0]])
        plt.title('gt')
        
        segmentation_types = list(history_segmentation[image_path].keys())
        for idx_seg, segmentation_type in enumerate(segmentation_types):
            segmentation = history_segmentation[image_path][segmentation_type]
            plt.subplot(num_rows, num_cols, idx_row * num_cols + 3 + idx_seg) 
            plt.axis('off') 
            plt.title(segmentation_type)
            plt.imshow(segmentation[:, :, [2, 1, 0]])
    plt.savefig('segmentation_history.png')

In [ ]:
show_segmentation_history()

# GRABCUT - interactive foreground extraction

In [ ]:
# https://www.docs.opencv.org/master/d8/d83/tutorial_py_grabcut.html
def run_grabcut(img):
    mask = np.zeros(img.shape[:2], np.uint8)
    bgdModel = np.zeros((1,65), np.float64)
    fgdModel = np.zeros((1,65), np.float64)
    rect = cv.selectROI(img)
    cv.destroyAllWindows()
    cv.grabCut(img, mask, rect, bgdModel, fgdModel, 5, cv.GC_INIT_WITH_RECT)
    mask2 = np.where((mask == 2)|(mask == 0),0,1).astype('uint8')
    img = img * mask2[:, :, np.newaxis]
    return img, mask
    
img = cv.imread('dataset/2_21_s.bmp')
img_grabcut, mask = run_grabcut(img)
show_images(image_1=img_grabcut, name_1='img_grabcut') 

In [ ]:
img = cv.imread('dataset/14_18_s.bmp')
img_grabcut, mask = run_grabcut(img)
show_images(image_1=img_grabcut, name_1='img_grabcut') 
# newmask is the mask image I manually labelled
newmask = cv.imread('14_18_s_paint.bmp',0)
# wherever it is marked white (sure foreground), change mask=1
# wherever it is marked black (sure background), change mask=0
mask[newmask == 0] = 0
mask[newmask == 255] = 1
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)
mask, bgdModel, fgdModel = cv.grabCut(img, mask, None, bgdModel, fgdModel, 5, cv.GC_INIT_WITH_MASK)
mask = np.where((mask == 2)|(mask == 0), 0, 1).astype('uint8')
img_refined = img * mask[:, :, np.newaxis] 
show_images(image_1=img_refined, name_1='img_grabcut_refined') 